In [ ]:
import akantu as aka

In [ ]:
class Support:
    def __init__(self, elem_filter, fem, spatial_dimension, elemtype, ghost_type):
        self.elem_filter = elem_filter
        self.fem = fem
        self.spatial_dimension = spatial_dimension
        self.elemtype = elemtype
        self.ghost_type = ghost_type

class TensorField:
    def __init__(self, name, support):
        self.name = name
        self.support = support

    def eval(self, output):
        pass

    def getNbComponent(self):
        raise NotImplementedError

    def transpose(self):
        raise NotImplementedError

    def __mul__(self, f):
        return DotField(self, f)

class NodalTensorField(TensorField):
    def __init__(self, name, support):
        super().__init__(name, support)
        self.nodal_field = np.array([])

    def eval(self, output):
        self.support.fem.interpolateOnIntegrationPoints(
            self.nodal_field, output, self.nodal_field.size, self.support.elemtype,
            self.support.ghost_type
        )

class IntegrationPointTensorField(TensorField):
    def eval(self, output):
        raise NotImplementedError

class DotField(TensorField):
    def __init__(self, f1, f2):
        super().__init__(f1.name + "." + f2.name, f1.support)
        self.field1 = f1
        self.field2 = f2

    def eval(self, output):
        o1 = np.array([])
        o2 = np.array([])
        self.field1.eval(o1)
        self.field2.eval(o2)
        for i in range(len(output)):
            output[i] = o1[i] * o2[i]

class GradientOperator(TensorField):
    def eval(self, output):
        shapes_derivatives = self.support.fem.getShapesDerivatives(
            self.support.elemtype, self.support.ghost_type
        )
        output[:] = shapes_derivatives

class FieldIntegrator:
    @staticmethod
    def integrate(field, support):
        nb_element = len(support.elem_filter)
        nb_nodes_per_element = Mesh.getNbNodesPerElement(support.elemtype)
        res = np.zeros((nb_element, nb_nodes_per_element * support.spatial_dimension))

        nb_component = field.getNbComponent()

        field_eval = np.array([])
        field.eval(field_eval)

        support.fem.integrate(field_eval, res, nb_component, support.elemtype,
                              support.ghost_type, support.elem_filter)
        return res

In [ ]:
# Test variational_evaluate_field_on_support
u = NodalTensorField("disp", support)
u_d = np.array([])
u.eval(u_d)
print(u_d)

# Test variational_consistent_force
grad = GradientOperator(support)
stress = IntegrationPointTensorField("stress", support)
f = FieldIntegrator.integrate(stress * grad, support)
print(f)

# Test variational_stiffness_matrix
elastic_tensor = IntegrationPointTensorField("C", support)
f = FieldIntegrator.integrate(grad.transpose() * elastic_tensor * grad, support)
print(f)